# Lab 2 - Unsupervised learning




In [ ]:
!pip install --upgrade pip
!pip install --upgrade numpy==1.20.0
!pip install --upgrade pandas
!pip install --upgrade nltk
!pip install --upgrade scikit-learn
!pip install --upgrade joblib
!pip install num2words
!pip install contractions
!pip install -U gensim
!pip install --upgrade somoclu

%matplotlib inline
import gensim.downloader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached numpy-1.20.0-cp38-cp38-manylinux2010_x86_64.whl (15.4 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas 1.5.2 requires numpy>=1.20.3; python_version < "3.10", but you have numpy 1.20.0 which is incompatible.
cmdstanpy 1.0.8 requires numpy>=1.21, but you have numpy 1.20.0 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached numpy-1.24.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Attemptin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Import the Dataset

The dataset we will work with is based on the [Carnegie Mellon Movie Summary Corpus](http://www.cs.cmu.edu/~ark/personas/). This is a dataset with 22,301 movie descriptions. The description of each film consists of its title, one or more tags characterizing the genre of the film, and finally a summary of its plot. First we import the dataset (use the code as is, you don't need the csv file) into dataframe `df_data_1`:

In [ ]:
import pandas as pd

dataset_url = "https://drive.google.com/uc?export=download&id=1zo13kUAf-MDMPZmBDxq1FxWtZY01lsxD"
df_data_1 = pd.read_csv(dataset_url, sep='\t',  header=None, quoting=3)

In [ ]:
import numpy as np

team_seed_number = 77

movie_seeds_url = "https://drive.google.com/uc?export=download&id=1g6F4TCHrs2wgtdOk7D3gtONaeirNt_Vo"
df_data_2 = pd.read_csv(movie_seeds_url, header=None)

# επιλέγεται
my_index = df_data_2.iloc[team_seed_number,:].values

titles = df_data_1.iloc[:, [2]].values[my_index] # movie titles (string)
categories = df_data_1.iloc[:, [3]].values[my_index] # movie categories (string)
bins = df_data_1.iloc[:, [4]]
catbins = bins[4].str.split(',', expand=True).values.astype(float)[my_index] # movie categories in binary form (1 feature per category)
summaries =  df_data_1.iloc[:, [5]].values[my_index] # movie summaries (string)
corpus = summaries[:,0].tolist() # list form of summaries
corpus_df = pd.DataFrame(corpus) # dataframe version of corpus

## Implementation 1. Implementation of content based movie recommendation system
<img src="http://clture.org/wp-content/uploads/2015/12/Netflix-Streaming-End-of-Year-Posts.jpg" width="70%">

### Convert to TFIDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# create sparse tf_idf representation
vectorizer = TfidfVectorizer()
vectorizer.fit(corpus)
corpus_tf_idf_plain = vectorizer.transform(corpus)

In [ ]:
print(corpus_tf_idf_plain.shape)

(5000, 48226)


### Implementation of the recommendation system


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def content_recommender(target_movie, max_recommendations, corpus_tf_idf):

  cosine_similarities = cosine_similarity(corpus_tf_idf[target_movie],corpus_tf_idf)[0]

  sorted_ID_indeces = (-cosine_similarities).argsort()[:max_recommendations+1]

  print('\033[1m'+ 'Target movie:'+ '\033[0m')
  print("ID: " , target_movie)
  print("Categories: ", categories[target_movie][0])
  print("Title: ", titles[target_movie][0])
  print("Summary: ", corpus[target_movie])
  print()

  for order,rec_movie in enumerate(sorted_ID_indeces[1:]):
    print('\033[1m'+'Recommended movie '+ str(order+1) +':' '\033[0m')
    print("Similarity score: ", cosine_similarities[rec_movie])
    print("ID: ", rec_movie)
    print("Categories: ", categories[rec_movie][0])
    print("Title: ", titles[rec_movie][0])
    print("Summary: ", corpus[rec_movie])
    print()
  return sorted_ID_indeces[1:]

### Testing the raw dataset and non-optimized TfidfVectorizer

In [ ]:
print(corpus_tf_idf_plain.shape)

(5000, 48226)


We create the `common_words` function which gives us the common words between the summaries so we can see which words play a major role in selecting matches.

In [ ]:
import nltk

nltk.download('punkt') # χρειάζεται για το tokenizer
from nltk.tokenize import word_tokenize

def common_words(corpus_1, corpus_2):
  text1 = nltk.word_tokenize(corpus_1.lower())
  text2 = nltk.word_tokenize(corpus_2.lower())
  commons = set()
  commons = set(text1) & set(text2)
  return(commons)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
test_ids = [5,7,13,20,21,24,6,12,26,27,40]
plain_recomendation_ids = []
for id in test_ids:
  rec_ids = content_recommender(id, 5, corpus_tf_idf_plain)
  plain_recomendation_ids.append(rec_ids)
  print('-----------------------------------------------------------------------------')

Target movie:
ID:  5
Categories:  "Drama",  "Family Film",  "Fantasy",  "Science Fiction",  "World cinema"
Title:  Santa Claus
Summary:  On December 24, Santa makes preparations for his yearly journey at his Toyland castle in outerspace. He plays the organ while his children helpers from all over the world sing. Meanwhile, in hell, Lucifer instructs his chief demon Pitch to travel to Earth and turn the children of the world against Santa.  In a busy marketplace, Pitch attempts to convince five children to “make Santa Claus angry”: Lupita, a poor girl; Billy, the son of wealthy but negligent parents; and three troublemaking brothers. Pitch fails at convincing Lupita to steal a doll from a vendor but succeeds in convincing the brothers to break a shop window. Santa’s child workers alert him to these events. Unable to travel to Earth before nightfall on Christmas Eve, he instead uses equipment to watch Pitch and the children. One device allows him to view Lupita’s dream, induced by Pitch,

In [ ]:
print('-----------------------------------------------------------------------------')
for tid,rid in zip(test_ids, plain_recomendation_ids):
  print('\033[1m'+'Common words between movies: '+ str(tid)+","+str(rid[0]) +':' '\033[0m')
  print("Categories:", categories[tid][0])
  print("Categories:", categories[rid[0]][0])
  print( "Common words:",common_words(corpus[tid], corpus[rid[0]]) )
  print('-----------------------------------------------------------------------------')

-----------------------------------------------------------------------------
Common words between movies: 5,1264:
Categories: "Drama",  "Family Film",  "Fantasy",  "Science Fiction",  "World cinema"
Categories: "Christmas movie",  "Children's",  "Comedy",  "Family Film",  "Slapstick"
Common words: {'one', 'children', 'instead', 'on', 'attempts', 'chief', 'escape', 'but', 'been', ',', 'sleigh', 'back', 'she', 'that', 'eve', 'at', 'by', 'which', 'when', 'he', 'home', 'convincing', 'over', 'tree', 'people', 'with', 'be', 'from', 'into', 'him', 'girl', 'is', 'and', 'then', 'good', 'has', 'two', 'her', 'so', '.', 'to', 'who', 'claus', 'his', 'meanwhile', 'a', 'all', 'himself', 'the', 'they', 'says', "'s", 'house', 'santa', 'while', 'of', 'left', 'in', 'say', 'find', 'as', 'three', 'magic', 'christmas', 'for'}
-----------------------------------------------------------------------------
Common words between movies: 7,1840:
Categories: "Romantic drama",  "Comedy-drama",  "Indie"
Categories: 

First we see that the size of the vectorizer is 48,226 which is quite large.

We notice what are the common words between the movies that had a good and bad prediction. The common words are mostly articles, pronouns, punctuation marks or names which have no useful information about the movie script. So regardless of whether our prediction is good or not the common features of the descriptions are not what they should be. So we will need to edit our data accordingly.

### Optimization of TfidfVectorizer


#### Preprocessing:

We will then edit the text to remove words and symbols that do not help match movies. We choose the following actions and construct the corresponding functions which are called by `preprocessing_corpus` in the correct order to do the preprocessing:

1. Remove html tags.
2. Remove urls.
3. Substitution of abbreviated words.
4. Convert all characters to lowercase.
5. Removal of English stopwords.
6. Remove punctuation in text and within words.
7. Removal of special characters.
8. Removal of words often found in movie descriptions.
9. Removal of names (?).
10. Removal of Ranking Words.
6. Remove strings less than 2.
7. Lemmatization.

In [ ]:
import nltk

nltk.download('punkt') # χρειάζεται για το tokenizer
from nltk.tokenize import word_tokenize

nltk.download('stopwords') # κατεβάζουμε ένα αρχείο που έχει stopwords στα αγγλικά
from nltk.corpus import stopwords
import string

nltk.download('wordnet')
nltk.download('rslp')

nltk.download('names')
import num2words

from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

import contractions

import io

from bs4 import BeautifulSoup
import unicodedata
import re

# Μερικές συναρτήσεις προέρχονται από # https://michael-fuchs-python.netlify.app/2021/05/22/nlp-text-pre-processing-i-text-cleaning/#removing-extra-whitespaces

def remove_html_tags_func(text):
  return BeautifulSoup(text, 'html.parser').get_text()

def remove_url_func(text):
  return re.sub(r'https?://\S+|www\.\S+', '', text)

def remove_accented_chars_func(text):
  return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')

def remove_punctuation_func(text):
  return re.sub(r'[^a-zA-Z0-9]', ' ', text)

def remove_irr_char_func(text):
  return re.sub(r'[^a-zA-Z]', ' ', text)

def remove_extra_whitespaces_func(text):
  return re.sub(r'^\s*|\s\s*', ' ', text).strip()

def remove_small_words(text, length):
  return ' '.join(word for word in text.split() if len(word)>length)

def remove_unwanted_words(text, unwanted_words):
  words = nltk.word_tokenize(text)
  filtered_words = [word for word in words if word not in unwanted_words] # 3
  return (" ".join(filtered_words))

def lemmatization(text):
  wordnet_lemmatizer = WordNetLemmatizer()
  words = nltk.word_tokenize(text)
  lem_words = [wordnet_lemmatizer.lemmatize(word) for word in words]
  return (" ".join(lem_words))

def stammatization(text):
  porter_stemmer = PorterStemmer()
  words = nltk.word_tokenize(text)
  stem_words = [porter_stemmer.stem(word) for word in words]
  return (" ".join(stem_words))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Package names is already up-to-date!


We create a `preprocessing_corpus` function to process your summary of each movie and call it for each movie.

In [ ]:
#συνάρτηση προεπεξεργασίας
def preprocessing_corpus(input_corpus):
  output_corpus = input_corpus.lower()
  output_corpus = remove_html_tags_func(output_corpus)
  output_corpus = remove_url_func(output_corpus)
  # output_corpus = remove_accented_chars_func(output_corpus)
  output_corpus = contractions.fix(output_corpus)

  names = [name.lower() for name in nltk.corpus.names.words()]
  numbers = [num2words.num2words(n, ordinal=True) for n in range(100)]
  film_words = ['film', 'story', 'plot', 'main', 'character', 'title', 'set', 'start', 'tell']
  unwanted_words = stopwords.words('english') + list(string.punctuation) + names + numbers + film_words  # Φιλτράρισμα λέξεων χωρις ενδιαφέρον
  output_corpus = remove_unwanted_words(output_corpus,unwanted_words)

  output_corpus = remove_punctuation_func(output_corpus)
  output_corpus = remove_irr_char_func(output_corpus)
  output_corpus = remove_extra_whitespaces_func(output_corpus)
  output_corpus = remove_small_words(output_corpus, 2)


  output_corpus = lemmatization(output_corpus)
  # output_corpus = stammatization(output_corpus)


  # words = nltk.word_tokenize(document.lower()) # 1,2
  # filtered_words = [word for word in words if word not in unwanted_words] # 3
  # filtered_words = thorough_filter(filtered_words) # 3

  # wordnet_lemmatizer = WordNetLemmatizer()
  # lem_words = [wordnet_lemmatizer.lemmatize(word) for word in filtered_words]

  # porter_stemmer = PorterStemmer()
  # stemmed_words = [porter_stemmer.stem(wordnet_lemmatizer.lemmatize(word)) for word in filtered_words]

  # return (" ".join(lem_words))
  return output_corpus

In [ ]:
preprocessed_corpus = [preprocessing_corpus(corpa) for corpa in corpus] #προ-επεξεργασμένο corpus

Some examples from text processing:

In [ ]:
print('------------')
print("Original summary:",corpus[10])
print("Preprocessed summary:",preprocessed_corpus[10])
print('------------')
print("Original summary:",corpus[100])
print("Preprocessed summary:",preprocessed_corpus[100])
print('------------')
print("Original summary:",corpus[120])
print("Preprocessed summary:",preprocessed_corpus[120])
print('------------')


------------
Original summary: After the death of her mother, Carrie McLaughlin  has been living with her grandmother in Pittsburgh. When her grandmother needs to go to a nursing home, Carrie has to move in with her father Hank , a rancher in Wyoming, whom she hasn't seen since she was a baby. Initially reluctant to adapt to country living, Carrie soon meets Flicka, a beautiful black Mustang that previously had belonged to Carrie's cousin Katy , who asked Hank to look after Flicka when her father sold their own ranch. Flicka is wild and dangerous and, according to the ranchers, longs for Katy. However, when Carrie is attacked by a rattlesnake, Flicka saves her and the two form a bond. Carrie also meets Jake , an attractive ranch hand hoping to become a country singer, and Amy Walker , the proud and arrogant daughter of a neighbour. Although Jake and Carrie take an immediate liking to each other, there is instant animosity between Carrie and Amy, mainly because Amy also likes Jake. When

We note here that we have removed names and common words from the English language so international movie names and phrases will continue to influence our predictions. However, a fact that can be helpful as Bollywood or Japanesse movies are closer. Also, apparently the size of the vectorizer decreased as the number of words and characters decreased.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# create sparse tf_idf representation
vectorizer = TfidfVectorizer()
vectorizer.fit(preprocessed_corpus)
prepro_corpus_tf_idf = vectorizer.transform(preprocessed_corpus)

print("Vectorizer size without preprocessing:",corpus_tf_idf_plain.shape)
print("Vectorizer size after preprocessing:",prepro_corpus_tf_idf.shape)


Vectorizer size without preprocessing: (5000, 48226)
Vectorizer size after preprocessing: (5000, 39336)


### Testing the preprocessed dataset and non-optimized TfidVectorizer
Next we see how the pre-processing affected the results.

In [ ]:
prepro_recomendation_ids = []
# for id in test_ids:
for id in test_ids:
  rec_ids = content_recommender(id, 5, prepro_corpus_tf_idf)
  prepro_recomendation_ids.append(rec_ids)
  print('-----------------------------------------------------------------------------')

Target movie:
ID:  5
Categories:  "Drama",  "Family Film",  "Fantasy",  "Science Fiction",  "World cinema"
Title:  Santa Claus
Summary:  On December 24, Santa makes preparations for his yearly journey at his Toyland castle in outerspace. He plays the organ while his children helpers from all over the world sing. Meanwhile, in hell, Lucifer instructs his chief demon Pitch to travel to Earth and turn the children of the world against Santa.  In a busy marketplace, Pitch attempts to convince five children to “make Santa Claus angry”: Lupita, a poor girl; Billy, the son of wealthy but negligent parents; and three troublemaking brothers. Pitch fails at convincing Lupita to steal a doll from a vendor but succeeds in convincing the brothers to break a shop window. Santa’s child workers alert him to these events. Unable to travel to Earth before nightfall on Christmas Eve, he instead uses equipment to watch Pitch and the children. One device allows him to view Lupita’s dream, induced by Pitch,

In [ ]:
print('-----------------------------------------------------------------------------')
for tid,rid in zip(test_ids, prepro_recomendation_ids):
  print('\033[1m'+'Common words between movies: '+ str(tid)+","+str(rid[0]) +':' '\033[0m')
  print("Categories:", categories[tid][0])
  print("Categories:", categories[rid[0]][0])
  print( "Common words:",common_words(preprocessed_corpus[tid], preprocessed_corpus[rid[0]]) )
  print('-----------------------------------------------------------------------------')

-----------------------------------------------------------------------------
Common words between movies: 5,1264:
Categories: "Drama",  "Family Film",  "Fantasy",  "Science Fiction",  "World cinema"
Categories: "Christmas movie",  "Children's",  "Comedy",  "Family Film",  "Slapstick"
Common words: {'one', 'instead', 'chief', 'call', 'escape', 'sleigh', 'back', 'attempt', 'home', 'convincing', 'tree', 'people', 'steal', 'girl', 'good', 'two', 'meanwhile', 'child', 'house', 'santa', 'left', 'say', 'find', 'three', 'magic', 'christmas'}
-----------------------------------------------------------------------------
Common words between movies: 7,4510:
Categories: "Romantic drama",  "Comedy-drama",  "Indie"
Categories: "Short Film"
Common words: {'life', 'filmmaker'}
-----------------------------------------------------------------------------
Common words between movies: 13,3260:
Categories: "Action"
Categories: "Action",  "Comedy"
Common words: {'fall', 'one', 'tell', 'life', 'rescue', 'b

We notice that the words in common between the first sentence and the target movie are significantly more relevant to the premise of the movie. So we expect our proposals to be more relevant.

### We select the TfidfVectorizer parameters:

1. `max_df = 0.2`: This parameter removes words that appear in one of the 10 crime descriptions.
2. `min_df = 10`: With this parameter words that appear only a few times in total in the texts are removed.

In [ ]:
max_df = 0.20
vectorizer = TfidfVectorizer(max_df = max_df)
vectorizer.fit(preprocessed_corpus)
opt1_prepro_corpus_tf_idf = vectorizer.transform(preprocessed_corpus)

print("Vectorizer size without preprocessing:",corpus_tf_idf_plain.shape)
print("Vectorizer size after preprocessing:",prepro_corpus_tf_idf.shape)
print("Vectorizer size with max_df=",max_df,":",opt1_prepro_corpus_tf_idf.shape)

Vectorizer size without preprocessing: (5000, 48226)
Vectorizer size after preprocessing: (5000, 39336)
Vectorizer size with max_df= 0.2 : (5000, 39304)


We notice that the size did not decrease too much. This is because we have already removed several common words during preprocessing. As we can see, some very common verbs and adverbs and nouns are mainly removed from the `stop_words_` of the vectorizer.

In [ ]:
vectorizer.stop_words_

{'also',
 'away',
 'back',
 'begin',
 'come',
 'day',
 'end',
 'fall',
 'family',
 'father',
 'find',
 'friend',
 'get',
 'go',
 'help',
 'home',
 'however',
 'later',
 'life',
 'make',
 'man',
 'meet',
 'new',
 'one',
 'return',
 'take',
 'tell',
 'time',
 'try',
 'two',
 'year',
 'young'}

In [ ]:
min_df = 10
vectorizer = TfidfVectorizer(max_df = max_df, min_df = min_df)
vectorizer.fit(preprocessed_corpus)
opt2_prepro_corpus_tf_idf = vectorizer.transform(preprocessed_corpus)

print("Vectorizer size without preprocessing:",corpus_tf_idf_plain.shape)
print("Vectorizer size after preprocessing:",prepro_corpus_tf_idf.shape)
print("Vectorizer size with max_df=",max_df,":",opt1_prepro_corpus_tf_idf.shape)
print("Vectorizer size with max_df=",max_df,"min_df=",min_df,":",opt2_prepro_corpus_tf_idf.shape)

Vectorizer size without preprocessing: (5000, 48226)
Vectorizer size after preprocessing: (5000, 39336)
Vectorizer size with max_df= 0.2 : (5000, 39304)
Vectorizer size with max_df= 0.2 min_df= 10 : (5000, 6753)


We notice that the size here decreases dramatically. This means that there are many words that appear only a few times. As we can see from the `stop_words_` of the vectorizer mainly foreign words, names as well as some misspelled words like "criuse" are removed.

In [ ]:
vectorizer.stop_words_

{'occupancy',
 'hull',
 'breckenridge',
 'jeopardizes',
 'gruver',
 'lackluster',
 'blurring',
 'scouted',
 'ringo',
 'premiership',
 'locality',
 'copyrighted',
 'invigorates',
 'lour',
 'deux',
 'rokusuke',
 'hopper',
 'nito',
 'goddaughter',
 'kodachi',
 'occupy',
 'kerosene',
 'rodimus',
 'conceited',
 'gaffney',
 'opium',
 'roasted',
 'auschwitz',
 'vestment',
 'berts',
 'starfire',
 'marquez',
 'milien',
 'nauseous',
 'diwan',
 'frisco',
 'murad',
 'darkened',
 'availability',
 'raftings',
 'bikas',
 'cullen',
 'moumouni',
 'chombi',
 'unimaginable',
 'quackery',
 'recultant',
 'twitch',
 'defines',
 'misjudgments',
 'bandersnatch',
 'glensville',
 'accessdatesecond',
 'fingertip',
 'roccabruno',
 'skillfully',
 'nondetected',
 'vaile',
 'commenting',
 'headder',
 'bain',
 'crorepati',
 'portfolio',
 'cropping',
 'singhal',
 'gennosuke',
 'heffley',
 'sorbian',
 'duplicating',
 'intensifies',
 'kilworth',
 'ruchi',
 'yashoda',
 'imboca',
 'enwick',
 'urine',
 'vaz',
 'borrega',
 

### Testing the preprocessed dataset and optimized TfidVectorizer


We will see how these parameters affected the movie suggestions:

In [ ]:
opt1_prepro_recomendation_ids = []
for id in test_ids:
  rec_ids = content_recommender(id, 5, corpus_tf_idf_plain)
  plain_recomendation_ids.append(rec_ids[0])
  print('-----------------------------------------------------------------------------')

Target movie:
ID:  5
Categories:  "Drama",  "Family Film",  "Fantasy",  "Science Fiction",  "World cinema"
Title:  Santa Claus
Summary:  On December 24, Santa makes preparations for his yearly journey at his Toyland castle in outerspace. He plays the organ while his children helpers from all over the world sing. Meanwhile, in hell, Lucifer instructs his chief demon Pitch to travel to Earth and turn the children of the world against Santa.  In a busy marketplace, Pitch attempts to convince five children to “make Santa Claus angry”: Lupita, a poor girl; Billy, the son of wealthy but negligent parents; and three troublemaking brothers. Pitch fails at convincing Lupita to steal a doll from a vendor but succeeds in convincing the brothers to break a shop window. Santa’s child workers alert him to these events. Unable to travel to Earth before nightfall on Christmas Eve, he instead uses equipment to watch Pitch and the children. One device allows him to view Lupita’s dream, induced by Pitch,

In [ ]:
opt2_prepro_recomendation_ids = []
for id in test_ids:
# for id in range(100):
  rec_ids = content_recommender(id, 5, opt2_prepro_corpus_tf_idf)
  opt2_prepro_recomendation_ids.append(rec_ids)
  print('-----------------------------------------------------------------------------')

Target movie:
ID:  5
Categories:  "Drama",  "Family Film",  "Fantasy",  "Science Fiction",  "World cinema"
Title:  Santa Claus
Summary:  On December 24, Santa makes preparations for his yearly journey at his Toyland castle in outerspace. He plays the organ while his children helpers from all over the world sing. Meanwhile, in hell, Lucifer instructs his chief demon Pitch to travel to Earth and turn the children of the world against Santa.  In a busy marketplace, Pitch attempts to convince five children to “make Santa Claus angry”: Lupita, a poor girl; Billy, the son of wealthy but negligent parents; and three troublemaking brothers. Pitch fails at convincing Lupita to steal a doll from a vendor but succeeds in convincing the brothers to break a shop window. Santa’s child workers alert him to these events. Unable to travel to Earth before nightfall on Christmas Eve, he instead uses equipment to watch Pitch and the children. One device allows him to view Lupita’s dream, induced by Pitch,

## Comparison
The advantages of a recommender based on tfidf are that the size and time of processing and suggestions of the text is quite limited. Also removing very rare but also very common words enhances the selection of movies that have more common words related to the content. Nevertheless, there is still no perfect correlation of the content with its words as, as we will see in the following, some of the common words can function as a link but they can simply disorient our sentences.

### Cherry-picking

- Film 5 has 5 very good suggestions as the word Santa or Christmas is a good indicator for choosing Christmas seasonal tans.

- Movie 7 has very relevant movie suggestions about love and relationships. Especially the 4th sentence which is a James Bond movie. Probably the noun "Bond" here created confusion.

- Movie 13 has Indian-bollywood action movie related suggestions.

- Movie 20 has related sentences with short comedies.

- Movie 21 has related suggestions for movies in space. Here the common word "space" led to a good association of science fiction films.

- The movie 24 is a futuristic version of frankstein and has related suggestions for movies with frankstein and space.

### Nit-picking

- Movie 6 has a very apt first sentence which has a common theme around slavery in Africa. Afterwards, however, we receive proposals for films with a different content but which take place in Africa. Here the word "Africa" is not necessarily helpful.

- Movie 12 is about Alistair Little. This leads to the first 2 suggestions being for a "little boy" and "little bear" but then we have better suggestions.

- The film 26 has no sentences that do not have a good correlation but all have in their story the nucleus of a family and hence the words "family", "brother"

- The movie 27 has sentences which do not have a good correlation but they all have to do with "Texas" and "Houston" but in some sentences Houston is a name.

- The movie 40 has quite a few wrong suggestions as it has a very short synopsis.

## Deep learning: building corpora using word embeddings

### Gensim-data implants
Gensim includes several pre-trained Word2Vec implant models. With the next cell we get their list.

In [ ]:
import gensim.downloader
print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


We choose and download three of the models. To compare we chose two models with the same size `embending = 300` but different vocabulary and two with the same vocabulary but different size `embending = 50, 300`.

In [ ]:
pretrained_model1 = gensim.downloader.load('fasttext-wiki-news-subwords-300')
pretrained_model2 = gensim.downloader.load('glove-wiki-gigaword-50')
pretrained_model3 = gensim.downloader.load('glove-wiki-gigaword-300')

In [ ]:
print(len( pretrained_model1.index_to_key), pretrained_model1["yes"].shape)
print(len( pretrained_model2.index_to_key), pretrained_model2["yes"].shape)
print(len( pretrained_model3.index_to_key), pretrained_model3["yes"].shape)

999999 (300,)
400000 (50,)
400000 (300,)


### Generating implant-based corpora

To be able to integrate the knowledge present in the pre-trained implants into our own corpus we will proceed as described below.

For each movie description $d$, which consists of $N_d$ words $w_i$, the $tfidf$ of each word $w_i$ is given by the relation:

$$ tfidf(w_i) = tf(w_i,d) \cdot idf(w_i)$$

At the same time, each word $w_i$ corresponds to a vector $W2V(w_i)$ from the implant model we have introduced. The $W2V$ implant vectors will have dimension $m$ , depending on the model.

For each movie d, we can create a vector representation $W2V(d)$ of dimensions $m$ using $tfidf(w_i)$ as a weighting factor for each $W2V(w_i)$ implant:

$$ W2V(d) = \frac{tfidf(w_1)\cdot W2V(w_i) + tfidf(w_2)\cdot W2V(w_2) + \dotsc + tfidf(w_{N_{d}})\cdot W2V(w_ {N_{d}})}{tfidf(w_1)+tfidf(w_2)+ \dotsc + tfidf(w_{N_{d}})}$$

#### build_tfw2v
Following the above, we construct the function that calculates the W2V array for each corpus.

In [ ]:
from collections import Counter

def build_tfw2v(corpus, vectors, embeddings_size=300):

  skip_words = 0
  count_words = 0
  w2v_d = []
  for corpa  in corpus:
    corpus_words = nltk.word_tokenize(corpa)
    corpus_size = len(corpus_words)
    tf_wd_all = dict(Counter(corpus_words)) # how many time each word appears in one summary

    nominator = 0
    denominator = 0
    for word in tf_wd_all.keys():
      try:
        idf_w = vectorizer.idf_[vectorizer.vocabulary_[word]] # word's idf
        tf_wd = tf_wd_all[word]/corpus_size # word's tf
        w2v = vectors[word] # word's w2v
        nominator += tf_wd*idf_w*w2v
        denominator += tf_wd*idf_w
        count_words += 1
      except:
        skip_words += 1
    w2v_d.append(nominator/denominator)
  print('Skipped',skip_words, '/', count_words+skip_words)
  return np.array(w2v_d)


Then for each of the pre-trained models we call the function build_tfw2v.

In [ ]:
from scipy import sparse

%time W2V1 = build_tfw2v(preprocessed_corpus, pretrained_model1, embeddings_size=300)
prepro_corpus_w2v_tf_idf1 = sparse.csr_matrix(W2V1)

Skipped 111309 / 540261
CPU times: user 1min 33s, sys: 269 ms, total: 1min 33s
Wall time: 1min 33s


In [ ]:
%time W2V2 = build_tfw2v(preprocessed_corpus, pretrained_model2,embeddings_size=50)
prepro_corpus_w2v_tf_idf2 = sparse.csr_matrix(W2V2)

Skipped 111320 / 540261
CPU times: user 1min 30s, sys: 266 ms, total: 1min 30s
Wall time: 1min 30s


In [ ]:
%time W2V3 = build_tfw2v(preprocessed_corpus, pretrained_model3, embeddings_size=300)
prepro_corpus_w2v_tf_idf3 = sparse.csr_matrix(W2V3)

Skipped 111320 / 540261
CPU times: user 1min 33s, sys: 268 ms, total: 1min 33s
Wall time: 1min 33s


### Testing implant-based corpora.

Then we see how this method affected the first 5 sentences.

In [ ]:
w2v_prepro_recomendation_ids1 = []
for id in test_ids:
  rec_ids = content_recommender(id, 5, prepro_corpus_w2v_tf_idf1)
  w2v_prepro_recomendation_ids1.append(rec_ids)
  print('-----------------------------------------------------------------------------')

Target movie:
ID:  5
Categories:  "Drama",  "Family Film",  "Fantasy",  "Science Fiction",  "World cinema"
Title:  Santa Claus
Summary:  On December 24, Santa makes preparations for his yearly journey at his Toyland castle in outerspace. He plays the organ while his children helpers from all over the world sing. Meanwhile, in hell, Lucifer instructs his chief demon Pitch to travel to Earth and turn the children of the world against Santa.  In a busy marketplace, Pitch attempts to convince five children to “make Santa Claus angry”: Lupita, a poor girl; Billy, the son of wealthy but negligent parents; and three troublemaking brothers. Pitch fails at convincing Lupita to steal a doll from a vendor but succeeds in convincing the brothers to break a shop window. Santa’s child workers alert him to these events. Unable to travel to Earth before nightfall on Christmas Eve, he instead uses equipment to watch Pitch and the children. One device allows him to view Lupita’s dream, induced by Pitch,

In [ ]:
w2v_prepro_recomendation_ids2 = []
for id in test_ids:
  rec_ids = content_recommender(id, 5, prepro_corpus_w2v_tf_idf2)
  w2v_prepro_recomendation_ids2.append(rec_ids)
  print('-----------------------------------------------------------------------------')

Target movie:
ID:  5
Categories:  "Drama",  "Family Film",  "Fantasy",  "Science Fiction",  "World cinema"
Title:  Santa Claus
Summary:  On December 24, Santa makes preparations for his yearly journey at his Toyland castle in outerspace. He plays the organ while his children helpers from all over the world sing. Meanwhile, in hell, Lucifer instructs his chief demon Pitch to travel to Earth and turn the children of the world against Santa.  In a busy marketplace, Pitch attempts to convince five children to “make Santa Claus angry”: Lupita, a poor girl; Billy, the son of wealthy but negligent parents; and three troublemaking brothers. Pitch fails at convincing Lupita to steal a doll from a vendor but succeeds in convincing the brothers to break a shop window. Santa’s child workers alert him to these events. Unable to travel to Earth before nightfall on Christmas Eve, he instead uses equipment to watch Pitch and the children. One device allows him to view Lupita’s dream, induced by Pitch,

In [ ]:
w2v_prepro_recomendation_ids3 = []
for id in test_ids:
  rec_ids = content_recommender(id, 5, prepro_corpus_w2v_tf_idf3)
  w2v_prepro_recomendation_ids3.append(rec_ids)
  print('-----------------------------------------------------------------------------')

Target movie:
ID:  5
Categories:  "Drama",  "Family Film",  "Fantasy",  "Science Fiction",  "World cinema"
Title:  Santa Claus
Summary:  On December 24, Santa makes preparations for his yearly journey at his Toyland castle in outerspace. He plays the organ while his children helpers from all over the world sing. Meanwhile, in hell, Lucifer instructs his chief demon Pitch to travel to Earth and turn the children of the world against Santa.  In a busy marketplace, Pitch attempts to convince five children to “make Santa Claus angry”: Lupita, a poor girl; Billy, the son of wealthy but negligent parents; and three troublemaking brothers. Pitch fails at convincing Lupita to steal a doll from a vendor but succeeds in convincing the brothers to break a shop window. Santa’s child workers alert him to these events. Unable to travel to Earth before nightfall on Christmas Eve, he instead uses equipment to watch Pitch and the children. One device allows him to view Lupita’s dream, induced by Pitch,

## Analysis of results

In [ ]:
def make_markdown_table(array):
    """ the same input as above """

    nl = "\n"

    markdown = nl
    markdown += f"| {' | '.join(array[0])} |"

    markdown += nl
    markdown += f"| {' | '.join(['---']*len(array[0]))} |"

    markdown += nl
    for entry in array[1:]:
        markdown += f"| {' | '.join(entry)} |{nl}"

    return markdown

table = []
table_titles = ["Method", "Target", "Recomendation 1", "Recomendation 2", "Recomendation 3", "Recomendation 4", "Recomendation 5"]
table.append(table_titles)
methods = ["Plain TFIDF","Optimized TFIDF", "W2V - 1", "W2V - 2", "W2V - 3"]
recomendation_ids = [plain_recomendation_ids, opt2_prepro_recomendation_ids, w2v_prepro_recomendation_ids1, w2v_prepro_recomendation_ids2, w2v_prepro_recomendation_ids3]
for i in range(len(test_ids)):
  for method,rec_ids in zip(methods, recomendation_ids):
    line = [ method, categories[test_ids[i]][0]]
    for id in rec_ids[i]:
      line.append(categories[id][0])
    table.append(line)
  table.append(['','','','','','',''])
  table.append(['','','','','','',''])
  table.append(['','','','','','',''])

print(make_markdown_table(table))


| Method | Target | Recomendation 1 | Recomendation 2 | Recomendation 3 | Recomendation 4 | Recomendation 5 |
| --- | --- | --- | --- | --- | --- | --- |
| Plain TFIDF | "Drama",  "Family Film",  "Fantasy",  "Science Fiction",  "World cinema" | "Christmas movie",  "Children's",  "Comedy",  "Family Film",  "Slapstick" | "Children's Entertainment",  "Music",  "Christmas movie",  "Holiday Film" | "Romantic comedy",  "Children's/Family",  "Holiday Film",  "Christmas movie",  "Fantasy",  "Comedy",  "Family Film",  "Fantasy Comedy" | "Children's/Family",  "Holiday Film",  "Musical",  "Children's",  "Drama",  "Comedy",  "Family Film",  "Children's Fantasy" | "Children's/Family",  "Stop motion",  "Creature Film",  "Musical",  "Animation",  "Christmas movie",  "Fantasy",  "Children's Fantasy",  "Animated Musical",  "Family Film" |
| Optimized TFIDF | "Drama",  "Family Film",  "Fantasy",  "Science Fiction",  "World cinema" | "Christmas movie",  "Children's",  "Comedy",  "Family Film",  "Slapsti

| Method | Target | Recomendation 1 | Recomendation 2 | Recomendation 3 | Recomendation 4 | Recomendation 5 |
| --- | --- | --- | --- | --- | --- | --- |
| Plain TFIDF | "Drama",  "Family Film",  "Fantasy",  "Science Fiction",  "World cinema" | "Christmas movie",  "Children's",  "Comedy",  "Family Film",  "Slapstick" | "Children's Entertainment",  "Music",  "Christmas movie",  "Holiday Film" | "Romantic comedy",  "Children's/Family",  "Holiday Film",  "Christmas movie",  "Fantasy",  "Comedy",  "Family Film",  "Fantasy Comedy" | "Children's/Family",  "Holiday Film",  "Musical",  "Children's",  "Drama",  "Comedy",  "Family Film",  "Children's Fantasy" | "Children's/Family",  "Stop motion",  "Creature Film",  "Musical",  "Animation",  "Christmas movie",  "Fantasy",  "Children's Fantasy",  "Animated Musical",  "Family Film" |
| Optimized TFIDF | "Drama",  "Family Film",  "Fantasy",  "Science Fiction",  "World cinema" | "Christmas movie",  "Children's",  "Comedy",  "Family Film",  "Slapstick" | "Children's Entertainment",  "Music",  "Christmas movie",  "Holiday Film" | "Children's/Family",  "Holiday Film",  "Musical",  "Children's",  "Drama",  "Comedy",  "Family Film",  "Children's Fantasy" | "Romantic comedy",  "Children's/Family",  "Holiday Film",  "Christmas movie",  "Fantasy",  "Comedy",  "Family Film",  "Fantasy Comedy" | "Fantasy",  "Family Film",  "Drama" |
| W2V - 1 | "Drama",  "Family Film",  "Fantasy",  "Science Fiction",  "World cinema" | "Christmas movie",  "Children's",  "Comedy",  "Family Film",  "Slapstick" | "Romantic comedy",  "Children's/Family",  "Holiday Film",  "Christmas movie",  "Fantasy",  "Comedy",  "Family Film",  "Fantasy Comedy" | "Children's Entertainment",  "Music",  "Christmas movie",  "Holiday Film" | "Children's/Family",  "Stop motion",  "Creature Film",  "Musical",  "Animation",  "Christmas movie",  "Fantasy",  "Children's Fantasy",  "Animated Musical",  "Family Film" | "Computer Animation",  "Animation",  "Holiday Film",  "Children's/Family" |
| W2V - 2 | "Drama",  "Family Film",  "Fantasy",  "Science Fiction",  "World cinema" | "Children's Entertainment",  "Music",  "Christmas movie",  "Holiday Film" | "Romantic comedy",  "Children's/Family",  "Holiday Film",  "Christmas movie",  "Fantasy",  "Comedy",  "Family Film",  "Fantasy Comedy" | "Christmas movie",  "Children's",  "Comedy",  "Family Film",  "Slapstick" | "Western" | "Action/Adventure",  "Western",  "Action",  "Drama" |
| W2V - 3 | "Drama",  "Family Film",  "Fantasy",  "Science Fiction",  "World cinema" | "Children's Entertainment",  "Music",  "Christmas movie",  "Holiday Film" | "Romantic comedy",  "Children's/Family",  "Holiday Film",  "Christmas movie",  "Fantasy",  "Comedy",  "Family Film",  "Fantasy Comedy" | "Christmas movie",  "Children's",  "Comedy",  "Family Film",  "Slapstick" | "Children's/Family",  "Stop motion",  "Creature Film",  "Musical",  "Animation",  "Christmas movie",  "Fantasy",  "Children's Fantasy",  "Animated Musical",  "Family Film" | "Computer Animation",  "Animation",  "Holiday Film",  "Children's/Family" |
|  |  |  |  |  |  |  |
|  |  |  |  |  |  |  |
|  |  |  |  |  |  |  |
| Plain TFIDF | "Romantic drama",  "Comedy-drama",  "Indie" | "Romantic drama",  "Romance Film",  "Drama",  "Indie",  "Adventure" | "Musical",  "Comedy" | "Adventure",  "Black-and-white",  "Action/Adventure",  "Drama",  "Romance Film",  "Action" | "Drama",  "Black-and-white" | "Thriller",  "Action Thrillers",  "Adventure",  "Caper story",  "Action/Adventure",  "Mystery",  "Archives and records",  "Action" |
| Optimized TFIDF | "Romantic drama",  "Comedy-drama",  "Indie" | "Romantic comedy",  "Comedy-drama",  "Fantasy",  "Comedy",  "Time travel",  "Fantasy Comedy",  "Romantic fantasy" | "Romantic comedy",  "Romance Film",  "Comedy of manners",  "Comedy" | "Short Film" | "Thriller",  "Action/Adventure",  "Adventure",  "Action Thrillers",  "Glamorized Spy Film",  "Crime Fiction",  "Action",  "Spy" | "Romance Film",  "Drama" |
| W2V - 1 | "Romantic drama",  "Comedy-drama",  "Indie" | "Romantic comedy",  "Romance Film",  "Comedy" | "Romantic comedy",  "Romance Film",  "Comedy of manners",  "Comedy" | "Drama",  "Black-and-white" | "Bollywood",  "Musical",  "Comedy",  "World cinema" | "Romance Film",  "Drama",  "Comedy" |
| W2V - 2 | "Romantic drama",  "Comedy-drama",  "Indie" | "Romantic comedy",  "Romance Film",  "Comedy" | "Indie",  "Marriage Drama",  "Drama",  "Romantic drama",  "Romance Film",  "Feminist Film" | "Romantic comedy",  "Romance Film",  "Comedy of manners",  "Comedy" | "Romantic comedy",  "Indie",  "Family Drama",  "Comedy-drama",  "Drama",  "Comedy",  "Romance Film",  "Domestic Comedy" | "Drama" |
| W2V - 3 | "Romantic drama",  "Comedy-drama",  "Indie" | "Romantic comedy",  "Romance Film",  "Comedy" | "Romantic comedy",  "Romance Film",  "Comedy of manners",  "Comedy" | "Indie",  "Marriage Drama",  "Drama",  "Romantic drama",  "Romance Film",  "Feminist Film" | "Romantic comedy",  "Ensemble Film",  "Romance Film",  "Drama",  "Comedy" | "Romantic comedy",  "World cinema",  "Comedy-drama",  "Musical",  "Drama",  "Comedy",  "Romantic drama",  "Romance Film" |
|  |  |  |  |  |  |  |
|  |  |  |  |  |  |  |
|  |  |  |  |  |  |  |
| Plain TFIDF | "Action" | "Action",  "Comedy" | "World cinema",  "Romance Film",  "Drama",  "Art film",  "Bollywood" | "Thriller",  "Action Thrillers",  "World cinema",  "Action/Adventure",  "Action",  "Bollywood" | "Musical",  "Drama",  "Bollywood",  "World cinema" | "Crime Fiction",  "Thriller",  "Drama",  "Crime Drama" |
| Optimized TFIDF | "Action" | "Action",  "Comedy" | "Musical",  "Drama",  "Bollywood",  "World cinema" | "Thriller",  "Action Thrillers",  "World cinema",  "Action/Adventure",  "Action",  "Bollywood" | "World cinema",  "Musical",  "Comedy",  "Bollywood" | "Thriller",  "Action",  "Drama" |
| W2V - 1 | "Action" | "Action" | "Drama" | "Drama",  "Comedy" | "Action" | "Musical",  "World cinema",  "Bollywood" |
| W2V - 2 | "Action" | "Musical",  "World cinema",  "Bollywood" | "Action" | "Thriller",  "Crime Fiction",  "Action" | "Drama" | "Action" |
| W2V - 3 | "Action" | "Musical",  "World cinema",  "Bollywood" | "Action" | "Action" | "Action" | "Thriller",  "Crime Fiction" |
|  |  |  |  |  |  |  |
|  |  |  |  |  |  |  |
|  |  |  |  |  |  |  |
| Plain TFIDF | "Short Film",  "Comedy",  "Black-and-white" | "Short Film",  "Comedy",  "Black-and-white",  "Slapstick" | "Short Film",  "Comedy",  "Black-and-white" | "Short Film",  "Comedy",  "Black-and-white" | "Short Film",  "Comedy",  "Black-and-white" | "Short Film",  "Musical",  "Children's",  "Animation" |
| Optimized TFIDF | "Short Film",  "Comedy",  "Black-and-white" | "Action/Adventure",  "Western",  "Spaghetti Western",  "World cinema" | "Short Film",  "Comedy",  "Black-and-white",  "Slapstick" | "Romance Film",  "Action",  "Comedy",  "War film" | "Short Film",  "Comedy",  "Black-and-white" | "Short Film",  "Comedy",  "Black-and-white",  "Slapstick" |
| W2V - 1 | "Short Film",  "Comedy",  "Black-and-white" | "Action/Adventure",  "Western",  "Spaghetti Western",  "World cinema" | "Romance Film",  "Action",  "Comedy",  "War film" | "Short Film",  "Comedy",  "Black-and-white",  "Slapstick" | "Action",  "Drama",  "War film" | "Slapstick",  "Drama",  "Comedy",  "Black-and-white",  "Film adaptation" |
| W2V - 2 | "Short Film",  "Comedy",  "Black-and-white" | "Action",  "Drama",  "War film" | "Action/Adventure",  "Western",  "Spaghetti Western",  "World cinema" | "Adventure",  "Epic Western",  "Costume drama",  "Indian Western",  "Film adaptation",  "Epic",  "Action/Adventure",  "Revisionist Western",  "Western",  "Drama",  "Period piece",  "War film" | "Action/Adventure",  "Western",  "Cavalry Film",  "Black-and-white",  "War film" | "Action/Adventure",  "Action",  "Drama",  "Adventure" |
| W2V - 3 | "Short Film",  "Comedy",  "Black-and-white" | "Romance Film",  "Action",  "Comedy",  "War film" | "Action/Adventure",  "Western",  "Spaghetti Western",  "World cinema" | "Action",  "Drama",  "War film" | "Slapstick",  "Drama",  "Comedy",  "Black-and-white",  "Film adaptation" | "Drama",  "War film" |
|  |  |  |  |  |  |  |
|  |  |  |  |  |  |  |
|  |  |  |  |  |  |  |
| Plain TFIDF | "Science Fiction",  "Japanese Movies",  "Animation",  "Anime",  "Drama",  "Romance Film",  "Action" | "Time travel",  "Science Fiction",  "Action",  "Adventure" | "Science Fiction",  "Japanese Movies",  "Adventure",  "World cinema",  "Fantasy",  "Action" | "Drama",  "Art film",  "World cinema" | "Crime Fiction",  "Thriller",  "Drama",  "Crime Drama" | "Thriller",  "Drama",  "Adventure" |
| Optimized TFIDF | "Science Fiction",  "Japanese Movies",  "Animation",  "Anime",  "Drama",  "Romance Film",  "Action" | "Adventure",  "History",  "Docudrama",  "Period piece",  "Drama",  "Film adaptation" | "Action" | "Thriller",  "Science Fiction",  "Buddy film",  "Adventure",  "Comedy-drama",  "Action/Adventure",  "Drama",  "Comedy",  "Action" | "Thriller",  "Science Fiction",  "Japanese Movies",  "World cinema" | "Science Fiction",  "Japanese Movies",  "Adventure",  "World cinema",  "Fantasy",  "Action" |
| W2V - 1 | "Science Fiction",  "Japanese Movies",  "Animation",  "Anime",  "Drama",  "Romance Film",  "Action" | "Science Fiction",  "Adventure",  "Superhero movie",  "Action/Adventure",  "Family Film",  "Action" | "Indie",  "World cinema",  "Road movie",  "Drama",  "Biopic [feature]",  "Coming of age",  "Biographical film",  "Adventure" | "Adventure",  "History",  "Docudrama",  "Period piece",  "Drama",  "Film adaptation" | "Monster movie",  "Japanese Movies",  "World cinema" | "Science Fiction" |
| W2V - 2 | "Science Fiction",  "Japanese Movies",  "Animation",  "Anime",  "Drama",  "Romance Film",  "Action" | "Science Fiction",  "Adventure",  "Superhero movie",  "Action/Adventure",  "Family Film",  "Action" | "Japanese Movies" | "Science Fiction",  "Drama" | "Indie",  "World cinema",  "Road movie",  "Drama",  "Biopic [feature]",  "Coming of age",  "Biographical film",  "Adventure" | "Science Fiction",  "Animation" |
| W2V - 3 | "Science Fiction",  "Japanese Movies",  "Animation",  "Anime",  "Drama",  "Romance Film",  "Action" | "Adventure",  "History",  "Docudrama",  "Period piece",  "Drama",  "Film adaptation" | "Science Fiction",  "Adventure",  "Superhero movie",  "Action/Adventure",  "Family Film",  "Action" | "Indie",  "World cinema",  "Road movie",  "Drama",  "Biopic [feature]",  "Coming of age",  "Biographical film",  "Adventure" | "Science Fiction",  "Japanese Movies",  "Adventure",  "World cinema",  "Fantasy",  "Action" | "Science Fiction",  "Road movie",  "Fantasy",  "Romantic drama",  "Drama",  "Romance Film",  "Romantic fantasy" |
|  |  |  |  |  |  |  |
|  |  |  |  |  |  |  |
|  |  |  |  |  |  |  |
| Plain TFIDF | "Alien Film",  "Science Fiction",  "Fantasy",  "Horror",  "Creature Film" | "Drama",  "Science Fiction",  "Horror",  "Time travel",  "Monster" | "Short Film",  "Science Fiction",  "Family Film",  "Comedy",  "Animation" | "Science Fiction",  "Family Film",  "Comedy" | "Time travel",  "Science Fiction",  "Action",  "Adventure" | "Comedy film",  "Horror" |
| Optimized TFIDF | "Alien Film",  "Science Fiction",  "Fantasy",  "Horror",  "Creature Film" | "Drama",  "Science Fiction",  "Horror",  "Time travel",  "Monster" | "Science Fiction" | "Monster movie",  "Science Fiction",  "Japanese Movies",  "World cinema" | "Thriller",  "B-movie",  "Creature Film",  "Black-and-white",  "Monster",  "Horror",  "Costume Horror" | "Thriller",  "Crime Fiction",  "Japanese Movies",  "Action Thrillers",  "Action/Adventure",  "Drama",  "Comedy",  "Action" |
| W2V - 1 | "Alien Film",  "Science Fiction",  "Fantasy",  "Horror",  "Creature Film" | "Monster movie",  "Science Fiction",  "Japanese Movies",  "World cinema" | "Parody",  "Time travel",  "Action",  "Comedy",  "Absurdism" | "Drama",  "Science Fiction",  "Horror",  "Time travel",  "Monster" | "Time travel",  "Science Fiction",  "Action",  "Adventure" | "Science Fiction",  "Superhero movie",  "Superhero",  "Action/Adventure",  "Alien Film",  "Family Film",  "Action",  "Romance Film" |
| W2V - 2 | "Alien Film",  "Science Fiction",  "Fantasy",  "Horror",  "Creature Film" | "Drama",  "Science Fiction",  "Horror",  "Time travel",  "Monster" | "Monster movie",  "Science Fiction",  "Japanese Movies",  "World cinema" | "Science Fiction",  "Horror",  "Z movie",  "Black-and-white",  "Alien Film",  "Monster" | "Science Fiction",  "Adventure",  "Superhero movie",  "Superhero",  "Action/Adventure",  "Fantasy",  "Action" | "Time travel",  "Science Fiction",  "Action",  "Adventure" |
| W2V - 3 | "Alien Film",  "Science Fiction",  "Fantasy",  "Horror",  "Creature Film" | "Drama",  "Science Fiction",  "Horror",  "Time travel",  "Monster" | "Monster movie",  "Science Fiction",  "Japanese Movies",  "World cinema" | "Science Fiction",  "Creature Film" | "Science Fiction",  "Horror",  "Z movie",  "Black-and-white",  "Alien Film",  "Monster" | "Time travel",  "Science Fiction",  "Action",  "Adventure" |
|  |  |  |  |  |  |  |
|  |  |  |  |  |  |  |
|  |  |  |  |  |  |  |
| Plain TFIDF | "History",  "Indie",  "Documentary" | "History",  "Historical fiction",  "Mystery",  "Period piece",  "Drama",  "Costume drama",  "Courtroom Drama" | "Culture & Society",  "History",  "Political cinema",  "Political drama",  "Social issues",  "Documentary" | "Thriller",  "Action/Adventure",  "Action",  "Adventure" | "Drama" | "Culture & Society",  "Indie",  "History",  "Drama",  "Comedy",  "Political cinema",  "Social issues",  "Documentary" |
| Optimized TFIDF | "History",  "Indie",  "Documentary" | "History",  "Historical fiction",  "Mystery",  "Period piece",  "Drama",  "Costume drama",  "Courtroom Drama" | "Crime Fiction",  "Mystery",  "Silent film",  "Drama" | "Thriller",  "Action/Adventure",  "Action",  "Adventure" | "Short Film" | "Road movie",  "Drama",  "Indie",  "World cinema",  "Childhood Drama" |
| W2V - 1 | "History",  "Indie",  "Documentary" | "History",  "Historical fiction",  "Mystery",  "Period piece",  "Drama",  "Costume drama",  "Courtroom Drama" | "Culture & Society",  "Biography",  "Sports",  "Drama",  "Social issues",  "Documentary" | "Crime Fiction",  "Drama",  "War film" | "Culture & Society",  "History",  "Political cinema",  "Political drama",  "Social issues",  "Documentary" | "Historical fiction",  "Drama" |
| W2V - 2 | "History",  "Indie",  "Documentary" | "Drama",  "World cinema",  "War film" | "Historical fiction",  "Drama" | "Drama",  "Television movie" | "Culture & Society",  "History",  "Political cinema",  "Political drama",  "Social issues",  "Documentary" | "Culture & Society",  "Biography",  "Sports",  "Drama",  "Social issues",  "Documentary" |
| W2V - 3 | "History",  "Indie",  "Documentary" | "Culture & Society",  "History",  "Political cinema",  "Political drama",  "Social issues",  "Documentary" | "Culture & Society",  "Biography",  "Sports",  "Drama",  "Social issues",  "Documentary" | "Drama",  "World cinema",  "Documentary" | "History",  "Historical fiction",  "Mystery",  "Period piece",  "Drama",  "Costume drama",  "Courtroom Drama" | "Historical fiction",  "Drama" |
|  |  |  |  |  |  |  |
|  |  |  |  |  |  |  |
|  |  |  |  |  |  |  |
| Plain TFIDF | "Thriller",  "Crime Fiction",  "Drama",  "Political drama",  "Indie" | "Short Film",  "Musical" | "Crime Fiction",  "Silent film",  "Indie",  "Black-and-white" | "Science Fiction",  "Fantasy Adventure",  "Fantasy",  "Adventure",  "Sword and sorcery films" | "Period piece",  "Drama",  "Psychological thriller",  "Docudrama",  "Political drama" | "Crime Fiction",  "Thriller",  "Drama",  "Crime Drama" |
| Optimized TFIDF | "Thriller",  "Crime Fiction",  "Drama",  "Political drama",  "Indie" | "Children's",  "Animation" | "Crime Fiction",  "Silent film",  "Indie",  "Black-and-white" | "Biographical film",  "Biography",  "Drama",  "Political drama",  "Period piece" | "Drama" | "Action/Adventure",  "Western" |
| W2V - 1 | "Thriller",  "Crime Fiction",  "Drama",  "Political drama",  "Indie" | "Drama" | "Biographical film",  "Biography",  "Drama",  "Political drama",  "Period piece" | "Crime Fiction",  "Gangster Film",  "Black-and-white",  "Drama",  "Pre-Code",  "Film noir" | "Ensemble Film",  "Period piece",  "Drama" | "Thriller",  "Tragedy",  "Drama",  "Indie",  "Adventure" |
| W2V - 2 | "Thriller",  "Crime Fiction",  "Drama",  "Political drama",  "Indie" | "Crime Fiction",  "Gangster Film",  "Black-and-white",  "Drama",  "Pre-Code",  "Film noir" | "Family Drama",  "Drama",  "World cinema",  "Bollywood",  "War film" | "Melodrama",  "Drama" | "Drama",  "Film noir",  "Crime Drama" | "Thriller",  "Action/Adventure",  "Action",  "Japanese Movies",  "World cinema" |
| W2V - 3 | "Thriller",  "Crime Fiction",  "Drama",  "Political drama",  "Indie" | "Crime Fiction",  "Gangster Film",  "Black-and-white",  "Drama",  "Pre-Code",  "Film noir" | "Biographical film",  "Biography",  "Drama",  "Political drama",  "Period piece" | "Drama" | "Thriller",  "Action/Adventure",  "Action",  "Japanese Movies",  "World cinema" | "Thriller",  "Crime Fiction" |
|  |  |  |  |  |  |  |
|  |  |  |  |  |  |  |
|  |  |  |  |  |  |  |
| Plain TFIDF | "Romantic comedy" | "Action",  "Drama" | "Period piece",  "Drama",  "Psychological thriller",  "Docudrama",  "Political drama" | "Drama",  "Comedy",  "World cinema" | "Bollywood",  "Musical",  "Drama",  "Comedy",  "World cinema" | "World cinema" |
| Optimized TFIDF | "Romantic comedy" | "Thriller",  "Mystery",  "Action",  "Drama" | "Family Film" | "Drama" | "Drama",  "Comedy" | "Biographical film",  "Family Drama",  "Drama",  "War film" |
| W2V - 1 | "Romantic comedy" | "Romance Film" | "Thriller",  "Crime Thriller",  "Mystery",  "Marriage Drama",  "Crime Fiction",  "Film adaptation",  "Drama",  "Romantic drama",  "Romance Film" | "Crime Fiction",  "Thriller",  "Drama",  "Mystery" | "Drama" | "Drama",  "Comedy" |
| W2V - 2 | "Romantic comedy" | "Thriller",  "Crime Thriller",  "Mystery",  "Marriage Drama",  "Crime Fiction",  "Film adaptation",  "Drama",  "Romantic drama",  "Romance Film" | "Drama",  "World cinema" | "Crime Fiction",  "Thriller",  "Drama",  "Mystery" | "Romantic comedy",  "Thriller" | "Thriller",  "Horror",  "Supernatural",  "Psychological thriller",  "Short Film",  "Mystery",  "Suspense",  "Crime Thriller" |
| W2V - 3 | "Romantic comedy" | "Drama" | "Drama" | "Romance Film" | "Crime Fiction",  "Thriller",  "Drama",  "Mystery" | "Romantic comedy",  "Thriller" |
|  |  |  |  |  |  |  |
|  |  |  |  |  |  |  |
|  |  |  |  |  |  |  |
| Plain TFIDF | "Comedy film",  "Drama" | "Action/Adventure",  "Western" | "Drama",  "Social problem film",  "Indie" | "B-movie",  "Comedy",  "Horror",  "Slasher",  "Indie" | "Comedy" | "Documentary" |
| Optimized TFIDF | "Comedy film",  "Drama" | "Action/Adventure",  "Western" | "Family Drama",  "Drama",  "Comedy",  "World cinema" | "Thriller",  "Crime Fiction",  "Detective fiction",  "Detective",  "Mystery",  "Comedy",  "Slapstick" | "Drama",  "Social problem film",  "Indie" | "B-movie",  "Comedy",  "Horror",  "Slasher",  "Indie" |
| W2V - 1 | "Comedy film",  "Drama" | "Drama",  "Comedy" | "Period piece",  "Drama",  "Political drama" | "Biographical film",  "Biography",  "Drama",  "Musical",  "Biopic [feature]" | "Documentary" | "Crime Fiction",  "Parody",  "Comedy-drama",  "Drama",  "Comedy",  "Black comedy" |
| W2V - 2 | "Comedy film",  "Drama" | "Drama",  "Social problem film",  "Indie" | "Romantic comedy",  "Musical comedy",  "Musical",  "Drama",  "Comedy",  "Romance Film" | "Documentary" | "Music",  "Comedy film",  "Drama" | "Drama",  "Indie",  "Political drama" |
| W2V - 3 | "Comedy film",  "Drama" | "Documentary" | "Drama",  "Social problem film",  "Indie" | "Drama",  "Comedy",  "World cinema" | "Period piece",  "Drama",  "Political drama" | "Drama",  "Comedy" |
|  |  |  |  |  |  |  |
|  |  |  |  |  |  |  |
|  |  |  |  |  |  |  |
| Plain TFIDF | "Short Film",  "Animation" | "Musical",  "Children's",  "Animation" | "Costume drama",  "Epic",  "Period piece",  "Drama",  "Film adaptation",  "Romantic drama",  "Romance Film" | "Dance",  "Indie",  "Period piece",  "Musical",  "Coming of age",  "Drama",  "Romantic drama",  "Romance Film",  "Musical Drama",  "Teen" | "Silent film",  "Romance Film",  "Black-and-white",  "Comedy" | "Short Film",  "Family Film",  "Comedy",  "Animation" |
| Optimized TFIDF | "Short Film",  "Animation" | "Musical",  "Drama" | "Dance",  "Indie",  "Period piece",  "Musical",  "Coming of age",  "Drama",  "Romantic drama",  "Romance Film",  "Musical Drama",  "Teen" | "Romance Film" | "Horror" | "Courtroom Drama",  "Drama",  "Television movie" |
| W2V - 1 | "Short Film",  "Animation" | "Musical",  "Comedy" | "Family Drama",  "Drama",  "World cinema",  "Bollywood",  "War film" | "World cinema",  "Bollywood" | "Drama",  "Comedy" | "Computer Animation",  "Family Film",  "Fantasy",  "Adventure" |
| W2V - 2 | "Short Film",  "Animation" | "Romance Film",  "Drama",  "Comedy" | "Romance Film",  "Action" | "Horror" | "Silent film",  "Race movie",  "Drama",  "Indie",  "Black-and-white" | "Adventure",  "Epic",  "Black-and-white",  "Period piece",  "Drama",  "Film adaptation",  "Romantic drama",  "Romance Film" |
| W2V - 3 | "Short Film",  "Animation" | "Adventure",  "Epic",  "Black-and-white",  "Period piece",  "Drama",  "Film adaptation",  "Romantic drama",  "Romance Film" | "Romance Film" | "Musical",  "Comedy" | "Indie",  "World cinema",  "Family Drama",  "Drama",  "Comedy",  "Adventure" | "Drama" |
|  |  |  |  |  |  |  |
|  |  |  |  |  |  |  |
|  |  |  |  |  |  |  |

First we notice that the processing time is much longer than tfidf. This is partly due to the fact that our implementation is probably not optimal.

Also, although the vocabulary in both cases is almost half, in both cases the words we skip are not much more.

To compare the semantic results more easily we created the table which contains in each column the categories of the recommended movies for each method. We notice that the raw text tends to have very good first sentences but quite irrelevant subsequent sentences. By preprocessing and optimizing the vectorizer we have progress in the number of sentences that are accurate. However, using the pre-trained models we observe overall better results and especially for ``fasttext-wiki-news-subwords-300'' which has the largest vocabulary and embending.